# 1. In Spin Language

This model is given by a simple 2-site Hamiltonian 
$$
H = \frac{\omega}{2}(\sigma_1^z+\sigma_2^z)+ g\sigma_x^1 \sigma_x^2.
$$
The evolution with dissipation as defined in [Hartman](https://iopscience.iop.org/article/10.1088/1367-2630/9/7/230/pdf)
$$
\dot {\rho}(t)=-i[H,\rho(t)] +\sum_{i=1}^{2}-\frac{B}{2}(1-s)\left[\sigma_{+}^{(i)} \sigma_{-}^{(i)} \rho+\rho \sigma_{+}^{(i)} \sigma_{-}^{(i)}-2 \sigma_{-}^{(i)} \rho \sigma_{+}^{(i)}\right]\quad-\frac{B}{2} s\left[\sigma_{-}^{(i)} \sigma_{+}^{(i)} \rho+\rho \sigma_{-}^{(i)} \sigma_{+}^{(i)}-2 \sigma_{+}^{(i)} \rho \sigma_{-}^{(i)}\right]
$$

In [1]:
from IPython.display import Image
from qutip import *
import numpy as np
from numpy import linalg as LA
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual

L =1

def sigma_z(k):
    op_list = []
    if k<=2*L-1 and k>=0:
        for i in range(2*L):
            if i==k:
                op_list.append(sigmaz())
            else:
                op_list.append(qeye(2))
        op = Qobj(tensor(op_list))
        return op

    else:
        raise ValueError("Index out of range: Operator sigma_z_"+str(k)+" may not be casted in a system with "+str(2*L)+" spins. By convention first spin is labeled as 0.")

def sigma_x(k):
    op_list = []
    if k<=2*L-1 and k>=0:
        for i in range(2*L):
            if i==k:
                op_list.append(sigmax())
            else:
                op_list.append(qeye(2))
        op = Qobj(tensor(op_list))
        return op

    else:
        raise ValueError("Index out of range: Operator sigma_x_"+str(k)+" may not be casted in a system with "+str(2*L)+" spins. By convention first spin is labeled as 0.")

def sigma_p(k):
    op_list = []
    if k<=2*L-1 and k>=0:
        for i in range(2*L):
            if i==k:
                op_list.append(sigmap())
            else:
                op_list.append(qeye(2))
        op = Qobj(tensor(op_list))
        return op

    else:
        raise ValueError("Index out of range: Operator sigma_p_"+str(k)+" may not be casted in a system with "+str(2*L)+" spins. By convention first spin is labeled as 0.")

def sigma_m(k):
    op_list = []
    if k<=2*L-1 and k>=0:
        for i in range(2*L):
            if i==k:
                op_list.append(sigmam())
            else:
                op_list.append(qeye(2))
        op = Qobj(tensor(op_list))
        return op

    else:
        raise ValueError("Index out of range: Operator sigma_m_"+str(k)+" may not be casted in a system with "+str(2*L)+" spins. By convention first spin is labeled as 0.")

Let's set $\omega = g = 1$ to ilustrate the model

In [2]:
H = 0.5*(sigma_z(0)+sigma_z(1))+1*sigma_x(0)*sigma_x(1)
H

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[ 1.  0.  0.  1.]
 [ 0.  0.  1.  0.]
 [ 0.  1.  0.  0.]
 [ 1.  0.  0. -1.]]

In [3]:
E,V = H.eigenstates()
V

array([Quantum object: dims = [[2, 2], [1, 1]], shape = (4, 1), type = ket
Qobj data =
[[ 0.38268343]
 [ 0.        ]
 [ 0.        ]
 [-0.92387953]],
       Quantum object: dims = [[2, 2], [1, 1]], shape = (4, 1), type = ket
Qobj data =
[[ 0.        ]
 [-0.70710678]
 [ 0.70710678]
 [ 0.        ]],
       Quantum object: dims = [[2, 2], [1, 1]], shape = (4, 1), type = ket
Qobj data =
[[ 0.        ]
 [-0.70710678]
 [-0.70710678]
 [ 0.        ]],
       Quantum object: dims = [[2, 2], [1, 1]], shape = (4, 1), type = ket
Qobj data =
[[0.92387953]
 [0.        ]
 [0.        ]
 [0.38268343]]], dtype=object)

### Negativity method

In [4]:
def E_N_by_L(dm,AB,A):
    if len(AB)/(2*L) < 1:
        rho_ab = (dm).ptrace(sel = AB,sparse=False)
    if len(AB)/(2*L) == 1:
        rho_ab = Qobj(tensor([Qobj(np.zeros((2,2)))]*(2*L)))
        rho_ab += dm  #v*v.dag()
    elif len(AB)/(2*L) > 1:
        raise ValueError("Input error: partial trace must take a subsystem NOT larger than system.")

    rho_p = partial_transpose(rho_ab,A) #hand checked for 8*8 matrix

    del(rho_ab)
    lam = rho_p.eigenenergies()
    del(rho_p)
    lam_neg_sum = 0.0
    for r in range(len(lam)):
        lam_neg_sum += 0.5*(abs(lam[r])-lam[r])
    # print(lam_neg_sum)
    log_neg = np.log(2*lam_neg_sum+1)
    return log_neg/(2*L)

### Simulation (spin language)

In [12]:
AB = [0,1]
A =  [0,1]

def negativity(in_state,w,g,s,B,t_f,t_steps,disp):
    H = w*0.5*(sigma_z(0)+sigma_z(1))+g*sigma_x(0)*sigma_x(1)
    E,V = H.eigenstates()
#     V = [ket("00"),f_0.dag()*ket("00"),f_1.dag()*ket("00"),f_0.dag()*f_1.dag()*ket("00")]      # Eigenstates
    
    rho_0 = V[in_state]*V[in_state].dag()             # choose eigenstate
    
    c_ops = [B*(1-s)*sigma_m(0)/2,B*s*sigma_p(0)/2,B*(1-s)*sigma_m(1)/2,B*s*sigma_p(1)/2]         # Collapse operators affecting only first fermion

    time_array = np.linspace(0.0, t_f, t_steps)       # time array for numerical solver
    rho_t = mesolve(H, rho_0, time_array, c_ops)
    
    neg = []
    for state in rho_t.states:
        neg.append(E_N_by_L(state,AB,A))
    
    plt.plot(time_array, neg, label="state in = "+str(np.real(V[in_state].dag().full())));
    plt.xlabel('t');
    plt.ylabel(r'$E_N/L$');
    plt.legend();
    
    if disp == 1:
        np.set_printoptions(precision=2)
        print("The initial state is:")
        print(np.around(rho_t.states[0], decimals=1))
        print("The final state is:")
        print(np.around(rho_t.states[len(rho_t.states)-1], decimals=2))
        print("Final (log) negativity:"+str(neg[len(neg)-1]))
        print("Final negativity:"+str((2**neg[len(neg)-1]-1)/2))
        
# warnings.filterwarnings('ignore')
interact(negativity, w=widgets.FloatSlider(value=1,min=-5,max=5,step=0.1), g=widgets.FloatSlider(value=1,min=-5,max=5,step=0.1),s=widgets.FloatSlider(value=0,min=0,max=1,step=0.1),B=widgets.FloatSlider(value=1,min=0,max=5,step=0.1),t_f=widgets.FloatSlider(value=50,min=1,max=1000,step=0.1),t_steps=widgets.IntSlider(value=500,min=10,max=1000,step=10),in_state=widgets.IntSlider(value=2**L,min=0,max=4**L-1,step=1),disp=widgets.IntSlider(value=1,min=0,max=1,step=1));

interactive(children=(IntSlider(value=2, description='in_state', max=3), FloatSlider(value=1.0, description='w…

# 1. In Fermionic Language

This model is given by a simple 2-site free Hamiltonian 
$$
H = \omega(n_1+n_2)-\omega+g(c_1^\dagger c_2^\dagger+c_2c_1+c_1^\dagger c_2+c_2^\dagger c_1).
$$
The evolution with dissipation as defined in [Hartman](https://iopscience.iop.org/article/10.1088/1367-2630/9/7/230/pdf)
$$
\dot {\rho}(t)=-i[H,\rho(t)] +\sum_{i=1}^{2}-\frac{B}{2}(1-s)\left[\sigma_{+}^{(i)} \sigma_{-}^{(i)} \rho+\rho \sigma_{+}^{(i)} \sigma_{-}^{(i)}-2 \sigma_{-}^{(i)} \rho \sigma_{+}^{(i)}\right]\quad-\frac{B}{2} s\left[\sigma_{-}^{(i)} \sigma_{+}^{(i)} \rho+\rho \sigma_{-}^{(i)} \sigma_{+}^{(i)}-2 \sigma_{+}^{(i)} \rho \sigma_{-}^{(i)}\right]
$$

In [10]:
def id():
    op_list = []
    for i in range(2*L):
        op_list.append(qeye(2))
    op = Qobj(tensor(op_list))
    return op

def sigma_z(k):
    op_list = []
    if k<=2*L-1 and k>=0:
        for i in range(2*L):
            if i==k:
                op_list.append(sigmaz())
            else:
                op_list.append(qeye(2))
        op = Qobj(tensor(op_list))
        return op

    else:
        raise ValueError("Index out of range: Operator sigma_z_"+str(k)+" may not be casted in a system with "+str(2*L)+" spins. By convention first spin is labeled as 0.")

def c(k):
    if k<=2*L-1 and k>=0:
        op_list = []
        for i in range(2*L):
            if i==k:
                op_list.append(destroy(2))
            else:
                op_list.append(qeye(2))
        aux = Qobj(tensor(op_list))
        phase = Qobj(tensor([qeye(2)]*(2*L)))
        for i in range(0,k):
            phase = phase*sigma_z(i)
        op = phase*aux
        return op

    else:
        raise ValueError("Index out of range: Operator c_"+str(k)+" may not be casted in a system with "+str(2*L)+" spins. By convention first spin is labeled as 0.")

## Simulation (fermionic language)

In [13]:
AB = [0,1]
A =  [0,1]

def negativity(in_state,w,g, s,B,t_f,t_steps,disp):
    H = w*(c(0).dag()*c(0)+c(1).dag()*c(1))-w+g*(c(0).dag()*c(1).dag()+c(1)*c(0)+c(0).dag()*c(1)+c(1).dag()*c(0))
    
    E,V = H.eigenstates()      # Eigenstates
    
    rho_0 = V[in_state]*V[in_state].dag()             # choose eigenstate
#     rho_0 = (ket("10")+ket("01"))*(bra("10")+bra("01"))/2
    
    aa = -1j*np.pi*c(0).dag()*c(0)
    bb = 1j*np.pi*c(0).dag()*c(0)
    phase1 = aa.expm()
    phase2 = bb.expm()
    
    c_ops = [B*(1-s)*c(0)/2,B*s*c(0).dag()/2,B*(1-s)*phase2*c(1)/2,B*s*phase1*c(1).dag()/2]         # Collapse operators affecting only first fermion

    time_array = np.linspace(0.0, t_f, t_steps)       # time array for numerical solver
    rho_t = mesolve(H, rho_0, time_array, c_ops)
    
    neg = []
    for state in rho_t.states:
        neg.append(E_N_by_L(state,AB,A))
    
    plt.plot(time_array, neg, label="state in = "+str(np.real(V[in_state].dag().full())));
    plt.xlabel('t');
    plt.ylabel(r'$E_N/L$');
    plt.legend();
    
    if disp == 1:
        np.set_printoptions(precision=2)
        print("The initial state is:")
        print(np.around(rho_t.states[0], decimals=1))
        print("The final state is:")
        print(np.around(rho_t.states[len(rho_t.states)-1], decimals=2))
        print("Final (log) negativity:"+str(neg[len(neg)-1]))
        print("Final negativity:"+str((2**neg[len(neg)-1]-1)/2))

# warnings.filterwarnings('ignore')
interact(negativity, w=widgets.FloatSlider(value=1,min=-5,max=5,step=0.1), g=widgets.FloatSlider(value=1,min=-5,max=5,step=0.1),s=widgets.FloatSlider(value=0,min=0,max=1,step=0.1),B=widgets.FloatSlider(value=1,min=0,max=5,step=0.1),t_f=widgets.FloatSlider(value=50,min=1,max=1000,step=0.1),t_steps=widgets.IntSlider(value=500,min=10,max=1000,step=10),in_state=widgets.IntSlider(value=2**L,min=0,max=4**L-1,step=1),disp=widgets.IntSlider(value=1,min=0,max=1,step=1));

interactive(children=(IntSlider(value=2, description='in_state', max=3), FloatSlider(value=1.0, description='w…